In [1]:
"""
This notebook is used for model hyper-parameter searching.
Also, if this can also be used as a baseline training script.
"""
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib
import matplotlib.pyplot as plt
from pprint import pprint
from typing import Dict, List

In [2]:
import sys
sys.path.append("../")
# If this notebook file is not placed under in /notebook/ directory,
# adding directory "../" might not correly add the project directory.
# If adding "../" does not solve the importing problem, we need to setup 
# the directory mannually.
try:
    import constants
except ModuleNotFoundError:
    core_dir = input("Directory of core files >>> ")
    if not core_dir.endswith("/"):
        core_dir += "/"
    sys.path.append(core_dir)
    import constants

In [3]:
from core.tools.metrics import *
import core.tools.visualize as visualize
from core.tools.time_series import *
from core.tools.data_import import *
import core.tools.rnn_prepare as rnn_prepare
import core.tools.param_set_generator as param_set_generator

import core.models.stacked_lstm as stacked_lstm

import core.training.hps_methods as hps_methods

In [4]:
# data preparation phase.
pprint(constants.DATA_DIR)
choice = None
while choice is None or choice not in constants.DATA_DIR.keys():
    if choice is not None:
        print("Invalid data location received, try again...")
    choice = input("Select Dataset >>> ")

FILE_DIR = constants.DATA_DIR[choice]

print(f"Dataset chosen: {FILE_DIR}")

print("Avaiable configuration files found: ")
for cf in os.listdir("../hps_configs"):
    if cf.endswith("config.py"):
        print("\t" + cf)

config_name = input("Select config file >>> ")
if config_name.endswith(".py"):
    config_name = config_name[:-3]

exec(f"import hps_configs.{config_name} as config")

# print("Reading configuration file...")
# for att in dir(config):
#     if att.endswith("_config"):
#         print(f"\tLoading: {att}")
#         exec(f"globals().update(config.{att})")

{'a': '/Users/tianyudu/Documents/Academics/EconForecasting/AnnEconForecast/data/UNRATE.csv',
 'b': '/home/ec2-user/AnnEconForecast/data/UNRATE.csv',
 'c': '/home/ec2-user/AnnEconForecast/data/DEXCAUS.csv'}
Select Dataset >>> b
Dataset chosen: /home/ec2-user/AnnEconForecast/data/UNRATE.csv
Avaiable configuration files found: 
	ec2_config.py
	mac_config.py
	sample_config.py
Select config file >>> ec2_config


In [5]:
parameter_collection = param_set_generator.gen_hparam_set(config.main)

Total number of parameter sets generated: 400


In [6]:
pprint(parameter_collection[1])

{'LAGS': 3,
 'ORDER': 1,
 'PERIODS': 1,
 'TRAIN_RATIO': 0.8,
 'VAL_RATIO': 0.1,
 'clip_grad': None,
 'epochs': 300,
 'fig_path': '/home/ec2-user/ec2_hps/2018DEC20_02/model_figs/LAGS=3-epochs=300-num_neurons=(128, '
             '512)-learning_rate=0.03',
 'hparam_str': 'LAGS=3-epochs=300-num_neurons=(128, 512)-learning_rate=0.03',
 'learning_rate': 0.03,
 'model_path': '/home/ec2-user/ec2_hps/2018DEC20_02/saved_models/LAGS=3-epochs=300-num_neurons=(128, '
               '512)-learning_rate=0.03',
 'num_inputs': 1,
 'num_neurons': (128, 512),
 'num_outputs': 1,
 'num_time_steps': None,
 'report_periods': 10,
 'tensorboard_path': '/home/ec2-user/ec2_hps/2018DEC20_02/tensorboard/LAGS=3-epochs=300-num_neurons=(128, '
                     '512)-learning_rate=0.03'}


In [7]:
# Check something.

In [8]:
for (i, param) in enumerate(parameter_collection):
    print("================================================================")
    print(f"Executing hyper-parameter searching session [{i}/{len(parameter_collection) - 1}]...")
    print("Session Config:\n\t" + param["hparam_str"].replace("-", "\n\t"))
    start = datetime.now()
    hps_methods.individual_train(
        param=param,
        exec_core=stacked_lstm.exec_core,
        file_dir=FILE_DIR
    )
    print(f"Time taken for session [{i}]: {str(datetime.now() - start)}.")
print("Done.")

Executing hyper-parameter searching session [0/399]...
Session Config:
	LAGS=3
	epochs=300
	num_neurons=(128, 512)
	learning_rate=0.01
Dataset loaded.    
	Index type: datetime64[ns]    
	Data type: float64
StandardScaler applied, scaling based on the first 679 observations.
Total 845 observations generated.
Note: shape format: (num_obs, time_steps, num_inputs/outputs)
X shape = (845, 3, 1), y shape = (845, 1, 1)
Training and testing set generated,        
X_train shape: (675, 3, 1)        
y_train shape: (675, 1)        
X_test shape: (85, 3, 1)        
y_test shape: (85, 1)        
X_validation shape: (85, 3, 1)        
y_validation shape: (85, 1)
Resetting Tensorflow defalut graph...
Note: no gradient clipping is applied.            
If possible gradient exploding detected (e.g. nan loss), try use clip_grad.
Starting training session...
Training model...

Iteration [0], Training MSE 46.5154572; Validation MSE 53.9969940

Iteration [100], Training MSE 0.8781286; Validation MSE 0.5518

Time taken for session [5]: 0:00:08.153498.
Executing hyper-parameter searching session [6/399]...
Session Config:
	LAGS=3
	epochs=300
	num_neurons=(64, 128, 256)
	learning_rate=0.1
Dataset loaded.    
	Index type: datetime64[ns]    
	Data type: float64
StandardScaler applied, scaling based on the first 679 observations.
Total 845 observations generated.
Note: shape format: (num_obs, time_steps, num_inputs/outputs)
X shape = (845, 3, 1), y shape = (845, 1, 1)
Training and testing set generated,        
X_train shape: (675, 3, 1)        
y_train shape: (675, 1)        
X_test shape: (85, 3, 1)        
y_test shape: (85, 1)        
X_validation shape: (85, 3, 1)        
y_validation shape: (85, 1)
Resetting Tensorflow defalut graph...
Note: no gradient clipping is applied.            
If possible gradient exploding detected (e.g. nan loss), try use clip_grad.
Starting training session...
Training model...

Iteration [0], Training MSE 3557.5703125; Validation MSE 3972.4616699

Iteration [

Time taken for session [11]: 0:00:07.868800.
Executing hyper-parameter searching session [12/399]...
Session Config:
	LAGS=3
	epochs=300
	num_neurons=(512, 1024)
	learning_rate=0.01
Dataset loaded.    
	Index type: datetime64[ns]    
	Data type: float64
StandardScaler applied, scaling based on the first 679 observations.
Total 845 observations generated.
Note: shape format: (num_obs, time_steps, num_inputs/outputs)
X shape = (845, 3, 1), y shape = (845, 1, 1)
Training and testing set generated,        
X_train shape: (675, 3, 1)        
y_train shape: (675, 1)        
X_test shape: (85, 3, 1)        
y_test shape: (85, 1)        
X_validation shape: (85, 3, 1)        
y_validation shape: (85, 1)
Resetting Tensorflow defalut graph...
Note: no gradient clipping is applied.            
If possible gradient exploding detected (e.g. nan loss), try use clip_grad.
Starting training session...
Training model...

Iteration [0], Training MSE 1071.1661377; Validation MSE 1058.4197998

Iteration [

Time taken for [300] epochs:  0:00:07.467503
Final result (validation set):
Loss Summary:
	mae=0.6122828125953674
	mse=0.6558794975280762
	rmse=0.8098638653755188
	mape=2.876399517059326
Time taken for session [17]: 0:00:11.947251.
Executing hyper-parameter searching session [18/399]...
Session Config:
	LAGS=3
	epochs=300
	num_neurons=(512, 512, 1024)
	learning_rate=0.1
Dataset loaded.    
	Index type: datetime64[ns]    
	Data type: float64
StandardScaler applied, scaling based on the first 679 observations.
Total 845 observations generated.
Note: shape format: (num_obs, time_steps, num_inputs/outputs)
X shape = (845, 3, 1), y shape = (845, 1, 1)
Training and testing set generated,        
X_train shape: (675, 3, 1)        
y_train shape: (675, 1)        
X_test shape: (85, 3, 1)        
y_test shape: (85, 1)        
X_validation shape: (85, 3, 1)        
y_validation shape: (85, 1)
Resetting Tensorflow defalut graph...
Note: no gradient clipping is applied.            
If possible gra

Note: no gradient clipping is applied.            
If possible gradient exploding detected (e.g. nan loss), try use clip_grad.
Starting training session...
Training model...

Iteration [0], Training MSE 156471.9531250; Validation MSE 163789.4843750

Iteration [100], Training MSE 0.9109184; Validation MSE 0.5891658

Iteration [200], Training MSE 1.1733078; Validation MSE 0.7971541

Iteration [300], Training MSE 15.9609690; Validation MSE 16.2848148

Iteration [400], Training MSE 1.1474307; Validation MSE 0.9767490
Saving the model...
Time taken for [500] epochs:  0:00:04.238566
Final result (validation set):
Loss Summary:
	mae=2.9397928714752197
	mse=9.338078498840332
	rmse=3.0558269023895264
	mape=58.4535026550293
Time taken for session [23]: 0:00:08.535751.
Executing hyper-parameter searching session [24/399]...
Session Config:
	LAGS=3
	epochs=500
	num_neurons=(64, 128, 256)
	learning_rate=0.01
Dataset loaded.    
	Index type: datetime64[ns]    
	Data type: float64
StandardScaler appl

...

Note: no gradient clipping is applied.            
If possible gradient exploding detected (e.g. nan loss), try use clip_grad.
Starting training session...
Training model...

Iteration [0], Training MSE 4209.9101562; Validation MSE 9387.6103516

Iteration [100], Training MSE 0.9142624; Validation MSE 0.7300281

Iteration [200], Training MSE 0.8345901; Validation MSE 0.6328656
Saving the model...
Time taken for [300] epochs:  0:00:10.061839
Final result (validation set):
Loss Summary:
	mae=0.6440050005912781
	mse=0.6434627175331116
	rmse=0.8021612763404846
	mape=5.756252765655518
Time taken for session [240]: 0:00:15.032960.
Executing hyper-parameter searching session [241/399]...
Session Config:
	LAGS=18
	epochs=300
	num_neurons=(128, 512)
	learning_rate=0.03
Dataset loaded.    
	Index type: datetime64[ns]    
	Data type: float64
StandardScaler applied, scaling based on the first 679 observations.
Total 830 observations generated.
Note: shape format: (num_obs, time_steps, num_inputs/ou

Note: no gradient clipping is applied.            
If possible gradient exploding detected (e.g. nan loss), try use clip_grad.
Starting training session...
Training model...

Iteration [0], Training MSE 65207.3476562; Validation MSE 64608.5429688

Iteration [100], Training MSE 0.9594310; Validation MSE 0.8125855

Iteration [200], Training MSE 0.9651732; Validation MSE 0.7877425
Saving the model...
Time taken for [300] epochs:  0:00:12.140131
Final result (validation set):
Loss Summary:
	mae=0.662725567817688
	mse=0.8117414712905884
	rmse=0.9009669423103333
	mape=1.9777917861938477
Time taken for session [246]: 0:00:17.214965.
Executing hyper-parameter searching session [247/399]...
Session Config:
	LAGS=18
	epochs=300
	num_neurons=(64, 128, 256)
	learning_rate=0.3
Dataset loaded.    
	Index type: datetime64[ns]    
	Data type: float64
StandardScaler applied, scaling based on the first 679 observations.
Total 830 observations generated.
Note: shape format: (num_obs, time_steps, num_inpu

Note: no gradient clipping is applied.            
If possible gradient exploding detected (e.g. nan loss), try use clip_grad.
Starting training session...
Training model...

Iteration [0], Training MSE 16016.1640625; Validation MSE 10205.2353516

Iteration [100], Training MSE 1.3422359; Validation MSE 1.1705778

Iteration [200], Training MSE 0.9708866; Validation MSE 0.7866657
Saving the model...
Time taken for [300] epochs:  0:00:23.848657
Final result (validation set):
Loss Summary:
	mae=0.6346328258514404
	mse=0.7635343074798584
	rmse=0.87380450963974
	mape=2.455538749694824
Time taken for session [252]: 0:00:28.689986.
Executing hyper-parameter searching session [253/399]...
Session Config:
	LAGS=18
	epochs=300
	num_neurons=(512, 1024)
	learning_rate=0.03
Dataset loaded.    
	Index type: datetime64[ns]    
	Data type: float64
StandardScaler applied, scaling based on the first 679 observations.
Total 830 observations generated.
Note: shape format: (num_obs, time_steps, num_inputs/o

Note: no gradient clipping is applied.            
If possible gradient exploding detected (e.g. nan loss), try use clip_grad.
Starting training session...
Training model...

Iteration [0], Training MSE 3675712.7500000; Validation MSE 3639705.0000000

Iteration [100], Training MSE 0.9871248; Validation MSE 4.2463913

Iteration [200], Training MSE 0.9072409; Validation MSE 4.3511434
Saving the model...
Time taken for [300] epochs:  0:00:30.520361
Final result (validation set):
Loss Summary:
	mae=1.0603663921356201
	mse=4.323191165924072
	rmse=2.079228401184082
	mape=13.660775184631348
Time taken for session [258]: 0:00:35.798370.
Executing hyper-parameter searching session [259/399]...
Session Config:
	LAGS=18
	epochs=300
	num_neurons=(512, 512, 1024)
	learning_rate=0.3
Dataset loaded.    
	Index type: datetime64[ns]    
	Data type: float64
StandardScaler applied, scaling based on the first 679 observations.
Total 830 observations generated.
Note: shape format: (num_obs, time_steps, num

ResourceExhaustedError: /home/ec2-user/ec2_hps/2018DEC20_02/saved_models/LAGS=18-epochs=300-num_neurons=(512, 512, 1024)-learning_rate=0.3.data-00000-of-00001.tempstate10502968884769124202; No space left on device
	 [[node save/SaveV2 (defined at ../core/models/stacked_lstm.py:158)  = SaveV2[dtypes=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, ..., DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save/Const_0_0, save/SaveV2/tensor_names, save/SaveV2/shape_and_slices, OPTIMIZER/beta1_power/_55, OPTIMIZER/beta2_power/_57, OUTPUT_LAYER/BIAS/_59, OUTPUT_LAYER/BIAS/ADAM_OPTIMIZER/_61, OUTPUT_LAYER/BIAS/ADAM_OPTIMIZER_1/_63, OUTPUT_LAYER/WEIGHT/_65, OUTPUT_LAYER/WEIGHT/ADAM_OPTIMIZER/_67, OUTPUT_LAYER/WEIGHT/ADAM_OPTIMIZER_1/_69, rnn/multi_rnn_cell/cell_0/LSTM_CELL_0/bias/_71, rnn/multi_rnn_cell/cell_0/LSTM_CELL_0/bias/ADAM_OPTIMIZER/_73, rnn/multi_rnn_cell/cell_0/LSTM_CELL_0/bias/ADAM_OPTIMIZER_1/_75, rnn/multi_rnn_cell/cel